In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px black solid !important;
  color: black !important;
}
</style>

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# processing of text
import stanza
import spacy
from pyMorfologik import Morfologik

# models
import torch
import lazypredict
import xgboost

# setting display
# from IPython.display import HTML
# styles = [dict(selector="th", props=[("text-align", "center")])]
pd.options.display.max_colwidth = 300
pd.set_option("display.colheader_justify","left")
%matplotlib inline

In [3]:
# load polish model of spacy
spacy_pl = spacy.load('pl_spacy_model')

In [4]:
data_poleval_raw = pd.read_csv('converted_label_poleval.txt', 
                           error_bad_lines=False, sep=",")
data_github_raw = pd.read_csv('labeled_dataset.txt', 
                          error_bad_lines=False, sep="|")

data_poleval = data_poleval_raw.copy(deep=True)
data_github = data_github_raw.copy(deep=True)
datasets = [data_poleval, data_github]

In [5]:
for data in datasets:
    display(data.head())

,Comment,Kind of offensive language
0,"Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.",0
1,@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe,0
2,"@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.",0
3,@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0


,Comment,No,Yes,Kind of offensive language
0,"Dziwię się, że Niemcy działają opieszale w sprawie gwałtów przecież nie wyszli chyba z wprawy z radykalnych rozwiązań tzn. w obozach dla tych islamskich zwierząt instalować przewoźne komory gazowe i po problemie.",3,28,2
1,Przywalić atomem tej islamskiej dziczy. Merkel powinna być powieszona publicznie.,2,7,2
2,"Tu trzeba jasno powiedzieć,wracasz albo kulka.",0,40,2
3,umrzesz za tydzień,16,0,2
4,"wybic do nogi prawie wszystkich arabów , pozostawiając kilku jako eksponaty",1,1,2


In [6]:
data_github['Kind of offensive language'].value_counts().to_frame()\
    .rename_axis('Response', axis=1).style.set_properties(
        **{'color': 'black !important',
           'border': '1px black solid !important',
           'text-align': 'center'}
        ).set_table_styles([{
            'selector': 'th',
            'props': [('border', '1px black solid !important')]
        }])

Response,Kind of offensive language
1,2305
2,43


Due to only two response "1" and "2" we reject column 'no' and 'yes' because this columns don't give us necessary information.

In [7]:
data_github.drop(['No', 'Yes'], axis=1, inplace=True)

In [8]:
data_github.head()

,Comment,Kind of offensive language
0,"Dziwię się, że Niemcy działają opieszale w sprawie gwałtów przecież nie wyszli chyba z wprawy z radykalnych rozwiązań tzn. w obozach dla tych islamskich zwierząt instalować przewoźne komory gazowe i po problemie.",2
1,Przywalić atomem tej islamskiej dziczy. Merkel powinna być powieszona publicznie.,2
2,"Tu trzeba jasno powiedzieć,wracasz albo kulka.",2
3,umrzesz za tydzień,2
4,"wybic do nogi prawie wszystkich arabów , pozostawiając kilku jako eksponaty",2
